**Mateo Alexander**

**Homework 2**

**Natural Language Processing: QMSS 5067**

**Professor Patrick Houlihan**

**Homework Due 10/21/2024**

In [ ]:
"""
Natural Language Processing for the Social Sciences (GR5067)
HW#2

A positive and negative word list (Hu and Bing 2004) exists on CourseWorks (Files 🡪 hw 🡪 hw2 🡪 negative-words.txt and positive-words.txt).
Please create a function called gen_senti that Tokenizes arbitrary text and compares each token with the positive and negative lexicons of each dictionary and outputs the sentiment score, S.
Positive and negative words, pw and nw, count as a score of 1 and -1 respectively for each word matched.
The total count for pw and nw are pc and nc, respectively.  Each message sentiment, S, is normalized between -1 and 1.  Any text that does not any positive AND negative words would have to be ignored, and not scored. (60 points)

S= s=opc+ncws pc+nc
where ws={   1,  p_{w}$ -1,  n_{w}$ 
For example: Let us say the following sentence was an input into the function “The darkest hour is among us in this time of gloom, however, we will prevail!”.
Let’s pretend the negative words were darkest and gloom and positive words were prevail/

S = (-1 + -1 + 1) / 3 = -1/3 = -0.3333

Using the dataframe from lecture, the_data, column body, apply this function to each corpus and add a column called “simple_senti” (15 points)

Using vaderSentiment, apply the “compound” value of sentiment for each corpus in column body on a new column of the_data called “vader” (15 points)

Compute the mean, median and standard_deviations of both sentiment measures, “simple_senti” and “vader”  (10 points)

References:

Hu, Minqing, and Bing Liu. "Mining and summarizing customer reviews." Proceedings of the tenth ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2004.

https://pypi.org/project/vaderSentiment/
"""

In [1]:
!pip install vaderSentiment

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [3]:
import os
import pandas as pd
from multiprocessing import Pool
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import urllib.request

In [5]:
from utils import read_file, clean_txt

In [43]:
file_path_data = os.path.abspath('Natural Language Processing/Data')
print(file_path_data)

/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Natural Language Processing/Data


In [45]:
# Establish the directory
directory = '/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data'

In [47]:
file_path_positive = os.path.abspath('Natural Language Processing/positive-words.txt')
file_path_negative = os.path.abspath('Natural Language Processing/negative-words.txt')

print(file_path_positive)
print(file_path_negative)

/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Natural Language Processing/positive-words.txt
/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Natural Language Processing/negative-words.txt


In [49]:
with open('/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/positive-words.txt', 'r', encoding='ISO-8859-1') as f:
    positive_words = set(f.read().splitlines())

with open('/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/negative-words.txt', 'r', encoding='ISO-8859-1') as f:
    negative_words = set(f.read().splitlines())

In [51]:
# Create gen_senti function for tokenization and positive and negative counts
def gen_senti(text):
    tokens = text.split()
    positive_count = sum(1 for word in tokens if word in positive_words)
    negative_count = sum(1 for word in tokens if word in negative_words)
    
    if positive_count + negative_count == 0:
        return None

    score = (positive_count - negative_count) / (positive_count + negative_count)
    return score

In [53]:
# Function to process individual files and calculate sentiment
def process_file(file_path):
    try:
        # Read and clean file
        content = read_file(file_path)
        content = clean_txt(content)  # Clean the text using your utility function
        
        if len(content) > 0:
            # Perform sentiment analysis: Senti Score & Vader Score
            simple_senti_score = gen_senti(content)
            vader_analyzer = SentimentIntensityAnalyzer()
            vader_score = vader_analyzer.polarity_scores(content)['compound']

            # Return results as a dictionary (to be converted to a DataFrame row later)
            return {
                'filename': file_path,
                'body': content,
                'simple_senti': simple_senti_score,
                'vader': vader_score
            }
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [55]:
def file_crawler_parallel(path_in):
    files_to_process = []
    
    # Walk through the directory to collect files
    for root, dirs, files in os.walk(path_in):
        for name in files:
            if name.endswith(".txt"):
                file_path = os.path.join(root, name)
                print(f"Found file: {file_path}")
                files_to_process.append(file_path)
    
    print(f"Total files to process: {len(files_to_process)}")
    
    # Use multiprocessing Pool to process files in parallel, 4 processors are used
    with Pool(processes=4) as pool:
        results = pool.map(process_file, files_to_process)
    
    # Filter out None results and DataFrame conversion
    results = [r for r in results if r is not None]
    print(f"Total processed results: {len(results)}")
    
    return pd.DataFrame(results)

In [59]:
# Call the File Crawler (Parallel Processing)
the_data = file_crawler_parallel(directory)

Found file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/what_nj_thinks_of_hiking_minimum_wage_legalizing_p.html_122408526000.txt
Found file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking_122503385000.txt
Found file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/www.njhiking.com_122402870000.txt
Found file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking_122521654000.txt
Found file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/breakneck-ridge_122417693000.txt
Found file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking_122518734000.txt
Found file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking-trails_122530676000.txt
Found file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiki

In [61]:
analyzer = SentimentIntensityAnalyzer()

# Walk through the directory and all its subdirectories
for root, dirs, files in os.walk(directory):
    for filename in files:
        if filename.endswith(".txt"):  # Ensure we're only reading text files
            file_path = os.path.join(root, filename)

            # Open and read the file content
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                content = file.read()
                print(f"Processing file: {file_path}, Content length: {len(content)}")

                # If content is empty, do not use, skip them
                if len(content) == 0:
                    print(f"File {file_path} is empty, skipping.")
                    continue

                # Process the content that is full
                simple_senti_score = gen_senti(content)
                vader_score = analyzer.polarity_scores(content)['compound']

                # Create a new DataFrame row with the results
                new_row = pd.DataFrame({
                    'filename': [file_path],
                    'body': [content],
                    'simple_senti': [simple_senti_score],
                    'vader': [vader_score]
                })

                # Concatenate the new row with the main DataFrame
                the_data = pd.concat([the_data, new_row], ignore_index=True)

Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/what_nj_thinks_of_hiking_minimum_wage_legalizing_p.html_122408526000.txt, Content length: 2377
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking_122503385000.txt, Content length: 5182
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/www.njhiking.com_122402870000.txt, Content length: 4701
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking_122521654000.txt, Content length: 9198
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/breakneck-ridge_122417693000.txt, Content length: 34621
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking_122518734000.txt, Content length: 2166
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5

/tmp/ipykernel_3113/3752194476.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  the_data = pd.concat([the_data, new_row], ignore_index=True)


Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hikeitbaby.com_122513568000.txt, Content length: 1593
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/the-best-hiking-near-nyc_122446728000.txt, Content length: 3743
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/Hiking_122358358000.txt, Content length: 4333
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking.htm_122557494000.txt, Content length: 3396
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/englewood_122412777000.txt, Content length: 1655
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Data/Hiking/hiking_122553476000.txt, Content length: 2094
Processing file: /home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Natural Language Processing/Dat

In [63]:
# Simple Mean statistics
simple_mean = the_data['simple_senti'].mean()
simple_median = the_data['simple_senti'].median()
simple_std = the_data['simple_senti'].std()

In [65]:
# Vader statistics
vader_mean = the_data['vader'].mean()
vader_median = the_data['vader'].median()
vader_std = the_data['vader'].std()

In [67]:
# Printing the results
print(f"Simple Sentiment - Mean: {simple_mean}, Median {simple_median}, Standard Deviation: {simple_std}")
print(f"Vader Sentiment - Mean: {vader_mean}, Median {vader_median}, Standard Deviation: {vader_std}") 

Simple Sentiment - Mean: 0.4233701070981633, Median 0.5, Standard Deviation: 0.4942874956017682
Vader Sentiment - Mean: 0.8804911196911196, Median 0.98505, Standard Deviation: 0.26575469666429363


In [69]:
the_data

,filename,body,simple_senti,vader
0,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,what n j thinks of hiking minimum wage legaliz...,0.250000,0.9658
1,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,hiking nyc parks on twitter official website o...,0.769231,0.9976
2,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,nj hiking njhiking com njhiking com hiking in ...,0.058824,0.9993
3,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,hiking trips hiking vacations tours travel wit...,0.600000,0.9945
4,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,breakneck ridge hike the hudson valley hike th...,0.328502,1.0000
...,...,...,...,...
513,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,Get your Deep-Sea Fishing Trip Tickets | Davey...,NaN,0.7418
514,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,Global Fishing Watch |\nMap How To FAQ Tutoria...,0.800000,0.9834
515,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,"Fishing | Fishing Tackle, Supplies, & Equipmen...",0.413043,0.9999
516,/home/37b4d573-02a2-4c75-b69b-8f21f8c5d212/Nat...,Fishing Permits & License Endorsements | Washi...,NaN,0.9296
